In [1]:

import pandas as pd
import ast
import numpy as np

import lightgbm as lgb
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, root_mean_squared_error
from scipy.sparse import hstack, csr_matrix, issparse, lil_matrix, save_npz, load_npz
from sklearn.model_selection import KFold
from tabpfn import TabPFNRegressor


In [2]:
# Converter of string lists into Python lists
# (e.g. "['a', 'b', 'c']" → [a, b, c])
def parse_list_col(s):
    return ast.literal_eval(s)

# Converter of 'N.V.' to 0, so column is numeric
def parse_vintage(s):
    return 0 if s == 'N.V.' else int(s)


base_path = '..\..\..\..\data\main'


In [3]:
# Load the train and test splits

wines = pd.read_csv(
    f'{base_path}\\XWines_Full_100K_wines.csv', 
    usecols=['WineID', 'Type', 'Elaborate', 'ABV', 'Body', 'Acidity', 'RegionName', 'WineryName', 'Grapes','Harmonize','Country'],
    converters={
        'Grapes':    parse_list_col,
        'Harmonize': parse_list_col
    }
)
train = pd.read_csv(
    f'{base_path}\\trainset.csv', 
    usecols=['UserID', 'WineID', 'Rating', 'Date', 'Vintage'],
    parse_dates=['Date'],
    date_format=lambda s: pd.to_datetime(s),
    converters={'Vintage': parse_vintage}
)
test_uwarm_iwarm = pd.read_csv(
    f'{base_path}\\testset_warm_user_warm_item.csv', 
    usecols=['RatingID', 'UserID', 'WineID', 'Rating', 'Date', 'Vintage'],
    parse_dates=['Date'],
    date_format=lambda s: pd.to_datetime(s),
    converters={'Vintage': parse_vintage}
)
test_uwarm_icold = pd.read_csv(
    f'{base_path}\\testset_warm_user_cold_item.csv', 
    usecols=['RatingID', 'UserID', 'WineID', 'Rating', 'Date', 'Vintage'],
    parse_dates=['Date'],
    date_format=lambda s: pd.to_datetime(s),
    converters={'Vintage': parse_vintage}
)
test_ucold_iwarm = pd.read_csv(
    f'{base_path}\\testset_cold_user_warm_item.csv', 
    usecols=['RatingID', 'UserID', 'WineID', 'Rating', 'Date', 'Vintage'],
    parse_dates=['Date'],
    date_format=lambda s: pd.to_datetime(s),
    converters={'Vintage': parse_vintage}
)
test_ucold_icold = pd.read_csv(
    f'{base_path}\\testset_cold_user_cold_item.csv', 
    usecols=['RatingID', 'UserID', 'WineID', 'Rating', 'Date', 'Vintage'],
    parse_dates=['Date'],
    date_format=lambda s: pd.to_datetime(s),
    converters={'Vintage': parse_vintage}
)

In [4]:
# Drop target column for all datasets

X_train = train.drop(columns=['Rating'])
y_train = train['Rating']

X_test_uwarm_iwarm = test_uwarm_iwarm.drop(columns=['Rating'])
y_test_uwarm_iwarm = test_uwarm_iwarm['Rating']

X_test_uwarm_icold = test_uwarm_icold.drop(columns=['Rating'])
y_test_uwarm_icold = test_uwarm_icold['Rating']

X_test_ucold_iwarm = test_ucold_iwarm.drop(columns=['Rating'])
y_test_ucold_iwarm = test_ucold_iwarm['Rating']

X_test_ucold_icold = test_ucold_icold.drop(columns=['Rating'])
y_test_ucold_icold = test_ucold_icold['Rating']

In [5]:
# # Take a small sample of the training npz set
X_train = X_train.sample(frac=0.1, random_state=42)
y_train = y_train.iloc[X_train.index]

In [6]:
# Split trainset into smaller subsets
kf = KFold(n_splits=5, shuffle=True, random_state=42)
tabpfn_preds = np.zeros((X_train.shape[0],))

for train_idx, _ in kf.split(X_train):
    sample_idx = np.random.choice(train_idx, size=8192, replace=False)  # sample subset
    clf = TabPFNRegressor(device='cuda')  # or 'cpu', but very slow
    clf.fit(X_train.iloc[sample_idx], y_train.iloc[sample_idx])
    tabpfn_preds += clf.predict(X_train) / 5  # average predictions


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.13 GiB. GPU 0 has a total capacity of 4.00 GiB of which 0 bytes is free. Of the allocated memory 8.70 GiB is allocated by PyTorch, and 36.21 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# Load transformed data from npz
X_train_transformed = load_npz(f'{base_path}\\preprocessed\\X_train_transformed.npz')
X_val_transformed = load_npz(f'{base_path}\\preprocessed\\X_val_transformed.npz')
X_test_uwarm_iwarm_transformed = load_npz(f'{base_path}\\preprocessed\\X_test_uwarm_iwarm_transformed.npz')
X_test_uwarm_icold_transformed = load_npz(f'{base_path}\\preprocessed\\X_test_uwarm_icold_transformed.npz')
X_test_ucold_iwarm_transformed = load_npz(f'{base_path}\\preprocessed\\X_test_ucold_iwarm_transformed.npz')
X_test_ucold_icold_transformed = load_npz(f'{base_path}\\preprocessed\\X_test_ucold_icold_transformed.npz')

# Load target variables
y_train = pd.read_csv(f'{base_path}\\preprocessed\\y_train.csv')
y_val = pd.read_csv(f'{base_path}\\preprocessed\\y_val.csv')
y_test_uwarm_iwarm = pd.read_csv(f'{base_path}\\preprocessed\\y_test_uwarm_iwarm.csv')
y_test_uwarm_icold = pd.read_csv(f'{base_path}\\preprocessed\\y_test_uwarm_icold.csv')
y_test_ucold_iwarm = pd.read_csv(f'{base_path}\\preprocessed\\y_test_ucold_iwarm.csv')
y_test_ucold_icold = pd.read_csv(f'{base_path}\\preprocessed\\y_test_ucold_icold.csv')



In [ ]:
# Combine preprocessed data with TabPFN predictions
X_train_combined = hstack([X_train_transformed, csr_matrix(tabpfn_preds.reshape(-1, 1))])
X_val_combined = hstack([X_val_transformed, csr_matrix(tabpfn_preds.reshape(-1, 1))])
X_test_uwarm_iwarm_combined = hstack([X_test_uwarm_iwarm_transformed, csr_matrix(tabpfn_preds.reshape(-1, 1))])
X_test_uwarm_icold_combined = hstack([X_test_uwarm_icold_transformed, csr_matrix(tabpfn_preds.reshape(-1, 1))])
X_test_ucold_iwarm_combined = hstack([X_test_ucold_iwarm_transformed, csr_matrix(tabpfn_preds.reshape(-1, 1))])
X_test_ucold_icold_combined = hstack([X_test_ucold_icold_transformed, csr_matrix(tabpfn_preds.reshape(-1, 1))])


In [ ]:
# Use the TabPFN predictions as a feature in LGBM
lgb_model = lgb.LGBMRegressor(random_state=42)
lgb_model.fit(X_train_combined, y_train)
y_pred_uwarm_iwarm = lgb_model.predict(X_test_uwarm_iwarm_combined)
y_pred_uwarm_icold = lgb_model.predict(X_test_uwarm_icold_combined)
y_pred_ucold_iwarm = lgb_model.predict(X_test_ucold_iwarm_combined)
y_pred_ucold_icold = lgb_model.predict(X_test_ucold_icold_combined)



In [ ]:
# Evaluate

# Calculate MSE for each test set
mse_uwarm_iwarm = mean_squared_error(y_test_uwarm_iwarm, y_pred_uwarm_iwarm)
mse_uwarm_icold = mean_squared_error(y_test_uwarm_icold, y_pred_uwarm_icold)
mse_ucold_iwarm = mean_squared_error(y_test_ucold_iwarm, y_pred_ucold_iwarm)
mse_ucold_icold = mean_squared_error(y_test_ucold_icold, y_pred_ucold_icold)

# Calculate RMSE for each test set
rmse_uwarm_iwarm = root_mean_squared_error(y_test_uwarm_iwarm, y_pred_uwarm_iwarm)
rmse_uwarm_icold = root_mean_squared_error(y_test_uwarm_icold, y_pred_uwarm_icold)
rmse_ucold_iwarm = root_mean_squared_error(y_test_ucold_iwarm, y_pred_ucold_iwarm)
rmse_ucold_icold = root_mean_squared_error(y_test_ucold_icold, y_pred_ucold_icold)

# Calculate MAE for each test set
mae_uwarm_iwarm = mean_absolute_error(y_test_uwarm_iwarm, y_pred_uwarm_iwarm)
mae_uwarm_icold = mean_absolute_error(y_test_uwarm_icold, y_pred_uwarm_icold)
mae_ucold_iwarm = mean_absolute_error(y_test_ucold_iwarm, y_pred_ucold_iwarm)
mae_ucold_icold = mean_absolute_error(y_test_ucold_icold, y_pred_ucold_icold)

# Print the results
print(f'MSE for warm user warm item: {mse_uwarm_iwarm:.4f}')
print(f'RMSE for warm user warm item: {rmse_uwarm_iwarm:.4f}')
print(f'MAE for warm user warm item: {mae_uwarm_iwarm:.4f}')
print('-' * 50)
print(f'MSE for warm user cold item: {mse_uwarm_icold:.4f}')
print(f'RMSE for warm user cold item: {rmse_uwarm_icold:.4f}')
print(f'MAE for warm user cold item: {mae_uwarm_icold:.4f}')
print('-' * 50)
print(f'MSE for cold user warm item: {mse_ucold_iwarm:.4f}')
print(f'RMSE for cold user warm item: {rmse_ucold_iwarm:.4f}')
print(f'MAE for cold user warm item: {mae_ucold_iwarm:.4f}')
print('-' * 50)
print(f'MSE for cold user cold item: {mse_ucold_icold:.4f}')
print(f'RMSE for cold user cold item: {rmse_ucold_icold:.4f}')
print(f'MAE for cold user cold item: {mae_ucold_icold:.4f}')


In [ ]:
# Train plain LGBM model
lgb_model = lgb.LGBMRegressor(random_state=42)
lgb_model.fit(X_train_transformed, y_train)
y_pred_uwarm_iwarm = lgb_model.predict(X_test_uwarm_iwarm_transformed)
y_pred_uwarm_icold = lgb_model.predict(X_test_uwarm_icold_transformed)
y_pred_ucold_iwarm = lgb_model.predict(X_test_ucold_iwarm_transformed)
y_pred_ucold_icold = lgb_model.predict(X_test_ucold_icold_transformed)

# Evaluate

# Calculate MSE for each test set
mse_uwarm_iwarm = mean_squared_error(y_test_uwarm_iwarm, y_pred_uwarm_iwarm)
mse_uwarm_icold = mean_squared_error(y_test_uwarm_icold, y_pred_uwarm_icold)
mse_ucold_iwarm = mean_squared_error(y_test_ucold_iwarm, y_pred_ucold_iwarm)
mse_ucold_icold = mean_squared_error(y_test_ucold_icold, y_pred_ucold_icold)

# Calculate RMSE for each test set
rmse_uwarm_iwarm = root_mean_squared_error(y_test_uwarm_iwarm, y_pred_uwarm_iwarm)
rmse_uwarm_icold = root_mean_squared_error(y_test_uwarm_icold, y_pred_uwarm_icold)
rmse_ucold_iwarm = root_mean_squared_error(y_test_ucold_iwarm, y_pred_ucold_iwarm)
rmse_ucold_icold = root_mean_squared_error(y_test_ucold_icold, y_pred_ucold_icold)

# Calculate MAE for each test set
mae_uwarm_iwarm = mean_absolute_error(y_test_uwarm_iwarm, y_pred_uwarm_iwarm)
mae_uwarm_icold = mean_absolute_error(y_test_uwarm_icold, y_pred_uwarm_icold)
mae_ucold_iwarm = mean_absolute_error(y_test_ucold_iwarm, y_pred_ucold_iwarm)
mae_ucold_icold = mean_absolute_error(y_test_ucold_icold, y_pred_ucold_icold)

# Print the results
print(f'MSE for warm user warm item: {mse_uwarm_iwarm:.4f}')
print(f'RMSE for warm user warm item: {rmse_uwarm_iwarm:.4f}')
print(f'MAE for warm user warm item: {mae_uwarm_iwarm:.4f}')
print('-' * 50)
print(f'MSE for warm user cold item: {mse_uwarm_icold:.4f}')
print(f'RMSE for warm user cold item: {rmse_uwarm_icold:.4f}')
print(f'MAE for warm user cold item: {mae_uwarm_icold:.4f}')
print('-' * 50)
print(f'MSE for cold user warm item: {mse_ucold_iwarm:.4f}')
print(f'RMSE for cold user warm item: {rmse_ucold_iwarm:.4f}')
print(f'MAE for cold user warm item: {mae_ucold_iwarm:.4f}')
print('-' * 50)
print(f'MSE for cold user cold item: {mse_ucold_icold:.4f}')
print(f'RMSE for cold user cold item: {rmse_ucold_icold:.4f}')
print(f'MAE for cold user cold item: {mae_ucold_icold:.4f}')
